You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from one specific endpoint: 
	All launches: https://api.spacexdata.com/v3/launches
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	Aggregate Operation:
o	Perform a simple "aggregate" operation on the retrieved launch data.
o	Aggregation Logic: Calculate the total number of launches for each year.
o	Return a print with the number of aggregate launches for each year
3.	Control Parameters and Debugging:
o	Include a variable at the beginning of the script to define the API endpoint URL, making it easily modifiable: 
	API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval, number of launches retrieved, starting aggregation, aggregation complete, number of years aggregated, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + aggregation). Print this time after the aggregation operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting aggregated summary dictionary (year counts) from the aggregate operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting aggregated JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.


In [0]:
import requests
import json
import logging
import time
from datetime import datetime

# Define the API endpoint URL
API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def retrieve_launch_data():
    """Retrieve launch data from the SpaceX API."""
    try:
        logging.info("Starting data retrieval...")
        response = requests.get(API_ENDPOINT_URL, timeout=10)
        response.raise_for_status()  # Raise an exception for bad status codes
        launch_data = response.json()
        logging.info(f"Number of launches retrieved: {len(launch_data)}")
        return launch_data
    except requests.exceptions.RequestException as e:
        logging.error(f"Error retrieving data: {e}")
        return None

def aggregate_launch_data(launch_data):
    """Aggregate launch data by year."""
    if launch_data is None:
        return None
    try:
        logging.info("Starting aggregation...")
        aggregated_data = {}
        for launch in launch_data:
            launch_year = datetime.strptime(launch['launch_date_utc'], '%Y-%m-%dT%H:%M:%S.%fZ').year
            if launch_year in aggregated_data:
                aggregated_data[launch_year] += 1
            else:
                aggregated_data[launch_year] = 1
        logging.info(f"Aggregation complete. Number of years aggregated: {len(aggregated_data)}")
        return aggregated_data
    except Exception as e:
        logging.error(f"Error aggregating data: {e}")
        return None

def upload_aggregated_data(aggregated_data):
    """Upload aggregated data to httpbin.org."""
    if aggregated_data is None:
        return
    try:
        logging.info("Starting upload to httpbin...")
        json_data = json.dumps(aggregated_data)
        response = requests.post('https://httpbin.org/post', data=json_data)
        response.raise_for_status()  # Raise an exception for bad status codes
        logging.info(f"Upload outcome: {response.json()['data']}")
    except requests.exceptions.RequestException as e:
        logging.error(f"Error uploading data: {e}")

def main():
    start_time = time.time()
    launch_data = retrieve_launch_data()
    aggregated_data = aggregate_launch_data(launch_data)
    if aggregated_data is not None:
        print("Number of aggregate launches for each year:")
        for year, count in aggregated_data.items():
            print(f"{year}: {count}")
    code_execution_time = time.time() - start_time
    logging.info(f"Code execution time: {code_execution_time:.2f} seconds")
    upload_aggregated_data(aggregated_data)
    total_execution_time = time.time() - start_time
    logging.info(f"Total execution time: {total_execution_time:.2f} seconds")

if __name__ == "__main__":
    main()
